This notebook implements intersection of convex polygons based on my understanding of this paper:

https://doi.org/10.1016/0146-664X(82)90023-5

with a PDF accessible from:

https://www.cs.jhu.edu/~misha/Spring16/ORourke82.pdf

EDIT: Version 2 - updated the code to include some differences to the algorithm present in C code published here https://www.science.smith.edu/~jorourke/books/ftp.html and also other changes to make it work for special cases.

In [ ]:
import numpy as np
from tweakwcs.linearfit import build_fit_matrix

In [210]:
eps = 1e-12

class Vertex:
    def __init__(self, x, y):
        self._xy = [x, y]
        
    def __repr__(self):
        return f"<{self.x:.3g}, {self.y:.3g}>"
    
    @property
    def x(self):
        return self._xy[0]
    
    @x.setter
    def x(self, x):
        self._xy[0] = x
    
    @property
    def y(self):
        return self._xy[1]
    
    @y.setter
    def y(self, y):
        self._xy[1] = y
        
    def __iter__(self):
        for i in range(2):
            yield self._xy[i]
    
    def __getitem__(self, key):
        return self._xy[key]
    
    def __setitem__(self, key, value):
        self._xy[key] = value
    
    def __add__(self, other):
        return Vertex(self.x + other[0], self.y + other[1])
    
    def __iadd__(self, other):
        self._xy[0] += other[0]
        self._xy[1] += other[1]
        return self
    
    def __sub__(self, other):
        return Vertex(self.x - other[0], self.y - other[1])
    
    def __isub__(self, other):
        self._xy[0] -= other[0]
        self._xy[1] -= other[1]
        return self
    
    def __mul__(self, other):
        return Vertex(self.x * other, self.y * other)
    
    def __imul__(self, other):
        self._xy[0] *= other
        self._xy[1] *= other
        return self
    
    def dot(self, other):
        return self.x * other[0] + self.y * other[1]
    
    def cross(self, other):
        return self.x * other[1] - self.y * other[0]


def subtended(p, v1, v2):
    v1 = v1 - p
    v2 = v2 - p
    dot = v1.dot(v2)
    cross = v1.cross(v2)
    return np.rad2deg(np.arctan2(cross, dot))


def is_hp(p_, p, pt):
    return ((p - p_).cross(pt - p_) > 0)


def is_inside(pt, poly):
    theta = subtended(pt, poly[-1], poly[0])
    for v1, v2 in zip(poly[:-1], poly[1:]):
        theta += subtended(pt, v1, v2)
    return theta > 0


def make_ccw_poly(p, debug=True):
    # direction of transversing a polygon
    c = np.mean(list(map(list, p)), axis=0)
    if is_hp(p[0], p[1], Vertex(*c)):
        return p
    else:
        if debug:
            print("Reversing polygon direction (P)")
        return p[::-1]


def intersect(p, q, debug=True):
    
    def output_vertex(vertex, name):
        if (not poly or abs(vertex.x - poly[-1].x) > abs(vertex.x) * eps or
                        abs(vertex.y - poly[-1].y) > abs(vertex.y) * eps):
            if debug:
                print(f"Appending vertex {name}: {vertex}")
            poly.append(vertex)
    
    
    if np.allclose(list(p[0]), list(p[-1]), rtol=eps):
        if debug:
            print("Removing endpoint vertex. polygon must be open")
            
    p = make_ccw_poly(p, debug)
    q = make_ccw_poly(q, debug)
    
    if all(is_inside(pv, q) for pv in p):
        return p
    elif all(is_inside(qv, p) for qv in q):
        return q
    
    lenp = len(p)
    lenq = len(q)
    assert lenp > 2
    assert lenq > 2
    
    if debug:
        print(f"Input polygon P: {p}")
        print(f"Input polygon Q: {q}")

    ip = 0
    iq = 0

    first_intersect = None

    # output polygon
    poly = []
    inside = None #"P"

    pv_ = p[(ip - 1) % lenp]
    pv = p[ip % lenp]
    qv_ = q[(iq - 1) % lenq]
    qv = q[iq % lenq]

    first_k = -2
    for k in range(2 * (lenp + lenq)):
        intersect_vertex = None
        if debug:
            print(f"\n*** k: {k}  ip: {ip}  iq: {iq}")
            print(f"p: {pv}  p_: {pv_}")
            print(f"q: {qv}  q_: {qv_}")

        dp = pv - pv_
        dq = qv - qv_
        
        p_in_hpqdot = is_hp(qv_, qv, pv)
        q_in_hppdot = is_hp(pv_, pv, qv)

        # https://en.wikipedia.org/wiki/Line–line_intersection
        t = (pv_.x - qv_.x) * (qv_.y - qv.y) - (pv_.y - qv_.y) * (qv_.x - qv.x)
        u = (pv_.x - qv_.x) * (pv_.y - pv.y) - (pv_.y - qv_.y) * (pv_.x - pv.x)
        # d = (pv_.x - pv.x) * (qv_.y - qv.y) - (pv_.x - pv.y) * (qv_.x - qv.x)
        cross = dp.cross(dq)
        if cross < 0:
            t = -t
            u = -u
            d = -cross
        else:
            d = cross

        if debug:
            print(f"Intersection check: t={t:.5e},  u={u:.5e},  d={d:.5e}")

        if ((0 <= t <= d) and (0 <= u <= d)) and d > 0.0:
            if debug:
                print("Intersection detected between p and q")
            t = t / d
            u = u / d
            xi = pv_.x + dp.x * t
            yi = pv_.y + dp.y * t
            intersect_vertex = Vertex(xi, yi)
            if debug:
                print(f"Intersection point (xi, yi): {intersect_vertex}")

            if first_intersect is None:
                if debug:
                    print("* first intersection!")
                first_intersect = intersect_vertex
                first_k = k

            elif abs(first_intersect.x - xi) == 0 and abs(first_intersect.y - yi) == 0:
                if k != first_k + 1:
                    if debug:
                        print(f"* END: k={k} first_k={first_k} break loop - same intersection  new: "
                              f"{intersect_vertex}   -- first: {first_intersect}")
                    break
                else:
                    if debug:
                        print("Repeating first vertex. Resseting first_k")

                    first_k = k
                
            if p_in_hpqdot:
                inside = "P"
            elif q_in_hppdot:
                inside = "Q"
            
            if debug:
                print(f"Setting 'INSIDE': {repr(inside)}")

            output_vertex(intersect_vertex, 'intersect_vertex')
            
        if cross == 0.0 and not p_in_hpqdot and not q_in_hppdot:
            if debug:
                print("* ADVANCING - branch 0")

            if inside == "P":
                iq += 1
                qv_ = qv
                qv = q[iq % lenq]
                if debug:
                    print("* advancing Q - branch 0: "
                          f"iq={iq}, {iq} mod {lenq}={iq % lenq}, qv_: {qv_},  qv: {qv}")
            else:
                #output_vertex(pv, '(px, py)')
                ip += 1
                pv_ = pv
                pv = p[ip % lenp]
                if debug:
                    print("* advancing P - branch 0: "
                          f"ip={ip}, {ip} mod {lenp}={ip % lenp}, pv_: {pv_},  pv: {pv}")
  
        elif cross >= 0.0:
            if debug:
                print("* ADVANCING - branch 1")

            if q_in_hppdot:
                if inside == "P":
                    output_vertex(pv, '(px, py)')
                ip += 1
                pv_ = pv
                pv = p[ip % lenp]
                if debug:
                    print("* advancing P - branch 1: "
                          f"ip={ip}, {ip} mod {lenp}={ip % lenp}, pv_: {pv_},  pv: {pv}")

            else:
                if inside == "Q":
                    output_vertex(qv, '(qx, qy)')
                iq += 1
                qv_ = qv
                qv = q[iq % lenq]
                if debug:
                    print("* advancing Q - branch 1: "
                          f"iq={iq}, {iq} mod {lenq}={iq % lenq}, qv_: {qv_},  qv: {qv}")

        else:
            print("* ADVANCING - branch 2")

            if p_in_hpqdot:
                if inside == "Q":
                    output_vertex(qv, '(qx, qy)')
                iq += 1
                qv_ = qv
                qv = q[iq % lenq]
                if debug:
                    print("* advancing Q - branch 2: "
                          f"iq={iq}, {iq} mod {lenq}={iq % lenq}, qv_: {qv_},  qv: {qv}")
                    
            else:
                if inside == "P":
                    output_vertex(pv, '(px, py)')
                ip += 1
                pv_ = pv
                pv = p[ip % lenp]
                if debug:
                    print("* advancing P - branch 2: "
                          f"ip={ip}, {ip} mod {lenp}={ip % lenp}, pv_: {pv_},  pv: {pv}")
                
    return poly

In [219]:
p = [(0, 0), (1, 0), (1, 1), (0, 1)]
# p = [(1, 1), (0, 1), (0, 0), (1, 0)][::-1]  # To test CW->CCW conversion

# test 1
q = [(x + 0.25, y + 0.1) for x, y in p]

# test 2:
R = build_fit_matrix(45)
q = [tuple(np.dot(R, pk).tolist()) for pk in p]

# test 3 (flipped wrt X-axis):
q = [(i, -j) for i, j in p]

# test 4a (Q - triangle with a common edge):
q = [(0, 0), (1, 0), (0.5, 0.6)]

# test 4b (Q - triangle with a common short edge):
q = [(0.1, 0), (0.9, 0), (0.5, 0.6)]

# test 4c (Q - triangle with a common long edge):
q = [(-0.1, 0), (1.1, 0), (0.5, 0.6)]

# test 5 (Q inside P):
q = [(0.1, 0.1), (0.9, 0.1), (0.9, 0.9), (0.1, 0.9)]

# test 6 (P inside Q):
q = [(-0.1, -0.1), (1.1, -0.1), (1.1, 1.1), (-0.1, 1.1)]

# test 7 (Q is P reflected wrt origin):
q = [(-i, -j) for i, j in p]

######################################

p = [Vertex(*v) for v in p]
q = [Vertex(*v) for v in q]

intersect(p, q)

Input polygon P: [<0, 0>, <1, 0>, <1, 1>, <0, 1>]
Input polygon Q: [<0, 0>, <-1, 0>, <-1, -1>, <0, -1>]

*** k: 0  ip: 0  iq: 0
p: <0, 0>  p_: <0, 1>
q: <0, 0>  q_: <0, -1>
Intersection check: t=0.00000e+00,  u=0.00000e+00,  d=0.00000e+00
* ADVANCING - branch 0
* advancing P - branch 0: ip=1, 1 mod 4=1, pv_: <0, 0>,  pv: <1, 0>

*** k: 1  ip: 1  iq: 0
p: <1, 0>  p_: <0, 0>
q: <0, 0>  q_: <0, -1>
Intersection check: t=0.00000e+00,  u=1.00000e+00,  d=1.00000e+00
Intersection detected between p and q
Intersection point (xi, yi): <0, 0>
* first intersection!
Setting 'INSIDE': None
Appending vertex intersect_vertex: <0, 0>
* ADVANCING - branch 1
* advancing Q - branch 1: iq=1, 1 mod 4=1, qv_: <0, 0>,  qv: <-1, 0>

*** k: 2  ip: 1  iq: 1
p: <1, 0>  p_: <0, 0>
q: <-1, 0>  q_: <0, 0>
Intersection check: t=0.00000e+00,  u=0.00000e+00,  d=0.00000e+00
* ADVANCING - branch 0
* advancing P - branch 0: ip=2, 2 mod 4=2, pv_: <1, 0>,  pv: <1, 1>

*** k: 3  ip: 2  iq: 1
p: <1, 1>  p_: <1, 0>
q: <-1, 0>

[<0, 0>]